![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Callysto's Weekly Data Visualization


## World Children's Day

### Recommended Grade levels: 

### Instructions

Click "Cell" and select "Run All".

This will import the data and run all the code, so you can see this week's data visualization. Scroll back to the top after you’ve run the cells.

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don't need to do any coding to view the visualizations**.

The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer?
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

## Question



### Goal

### Background



## Gather

### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [ ]:
import pandas as pd
import plotly.express as px
try:
    import pycountry_convert as pc
except:
    !pip install pycountry_convert
    import pycountry_convert as pc
print("Libaries imported")

In [ ]:
percentage_pop = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/world-children's-day/percentageofpopchildren.csv", skiprows=4)
primary_school = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/data-viz-of-the-week/world-children's-day/outofprimaryschool.csv", skiprows=4)
del percentage_pop['Unnamed: 67']
del primary_school['Unnamed: 67']
display(percentage_pop, primary_school)

In [ ]:
def country_code_to_continent(country_code):
    mapping = pc.map_country_alpha3_to_country_name()
    try:
        country_name = mapping[country_code]
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return

print(country_code_to_continent('ABW'))

In [ ]:
country_codes = list(percentage_pop['Country Code'])
print(country_codes)

In [ ]:
for country in country_codes:
    try:
        percentage_pop.loc[percentage_pop['Country Code'] == country, 'Continent'] = country_code_to_continent(country)
    except:
        pass
percentage_pop

In [ ]:
pd.set_option('display.max_rows', None)
countries_without_label = percentage_pop[percentage_pop['Continent'].isna()].reset_index(drop=True)
temp = list(countries_without_label['Country Name'])
display(temp)
pd.reset_option('display.max_rows')

In [ ]:
percentage_pop = percentage_pop.dropna(subset=['Continent'])
percentage_pop